In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (16,10)

In [2]:
def read_csv(TRAIN_PATH):
    chunksize = 5_000_000 # batch size
    
    traintypes = {
        'fare': 'float32',
        'from': 'str',
        'to': 'str',
        'p_count': 'uint8', 
        'hour': 'uint8',
        'weekday': 'uint8',
        'week': 'uint8',
        'year': 'int16',
        'dist_e': 'float32',
        'dist_t': 'float32',
        'x0': 'float32',
        'x1': 'float32',
        'y0': 'float32',
        'y1': 'float32',
    }
    
    cols = list(traintypes.keys())
    
    # loading the dataframe into list of small dataframes
    df_list = []

    for df_chunk in pd.read_csv(TRAIN_PATH, usecols=cols, dtype=traintypes, chunksize=chunksize):
        df_list.append(df_chunk) 
        df = pd.concat(df_list)
        
    return df

In [3]:
train = read_csv('../input/train_cleaned_2.csv')

In [4]:
train.head()

,fare,from,to,hour,weekday,week,year,p_count,dist_e,dist_t,x0,x1,y0,y1
0,4.5,Queens,Queens,17,0,25,2009,1,0.638322,0.759079,-73.844315,-73.841614,40.721317,40.712276
1,16.9,Manhattan,Manhattan,16,1,1,2010,1,5.213395,6.717119,-74.016045,-73.979271,40.711304,40.782005
2,5.7,Manhattan,Manhattan,0,3,33,2011,2,0.852610,1.164240,-73.982735,-73.991241,40.761269,40.750561
3,7.7,Manhattan,Manhattan,4,5,16,2012,1,1.735600,1.943343,-73.987129,-73.991570,40.733143,40.758091
4,5.3,Manhattan,Manhattan,7,1,10,2010,1,1.228196,1.658726,-73.968094,-73.956657,40.768009,40.783764


In [5]:
y = train['fare']
X = train.drop(['fare'], axis=1)
del train

from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.25, random_state=379582)

del X,y

In [6]:
import tensorflow as tf

In [21]:
locations = [
    'Manhattan', 'JFK', 'LaGardia', 'Newark', 'Teterboro', 'Brooklin', 'Bronx', 'Staten', 'Queens', 'Westchester',
    'Nassau', 'Jersey', 'Bayonne', 'WStaton', 'West', 'WNewYork', 'North'        
]

# num_buckets = [0, num_buckets)

p_count = tf.feature_column.numeric_column('p_count')
taxicab = tf.feature_column.numeric_column('dist_t')
euclidean = tf.feature_column.numeric_column('dist_e')

x0 = tf.feature_column.numeric_column('x0')
x1 = tf.feature_column.numeric_column('x1')
y0 = tf.feature_column.numeric_column('y0')
y1 = tf.feature_column.numeric_column('y1')

numbuckets = 30
dx = np.linspace(-74.27, -72.98, numbuckets).tolist()
dy = np.linspace(40.56, 41.71, numbuckets).tolist()

x0_B = tf.feature_column.bucketized_column(x0, dx)
x1_B = tf.feature_column.bucketized_column(x1, dx)
y0_B = tf.feature_column.bucketized_column(y0, dy)
y1_B = tf.feature_column.bucketized_column(y1, dy)

xy_0 = tf.feature_column.crossed_column([x0_B, y0_B], numbuckets * numbuckets)
xy_1 = tf.feature_column.crossed_column([x1_B, y1_B], numbuckets * numbuckets)
xy_pair = tf.feature_column.crossed_column([xy_0, xy_1], numbuckets**4)

# definition of the categorical columns
hour = tf.feature_column.categorical_column_with_identity('hour', num_buckets = 24)
weekday = tf.feature_column.categorical_column_with_identity('weekday', num_buckets = 7)
week = tf.feature_column.categorical_column_with_identity('week', num_buckets = 54)
year = tf.feature_column.categorical_column_with_vocabulary_list('year', [2009, 2010, 2011, 2012, 2013, 2014, 2015])

loc_from = tf.feature_column.categorical_column_with_vocabulary_list('from', locations)
loc_to = tf.feature_column.categorical_column_with_vocabulary_list('to', locations)

# definition of the feature crosses
from_to = tf.feature_column.crossed_column([loc_from, loc_to], len(locations)**2)
from_to_year = tf.feature_column.crossed_column([from_to, year], len(locations)**2 * 7)
from_to_year_week = tf.feature_column.crossed_column([from_to_year, week], len(locations)**2 * 7 * 54)

weekday_year = tf.feature_column.crossed_column([weekday, year], 7 * 7)
hour_weekday_year = tf.feature_column.crossed_column([hour, weekday, year], 24 * 7 * 7)

In [22]:
deep_columns = [
    # Embedding_column to "group" together ...
    tf.feature_column.embedding_column(from_to, 50),
    tf.feature_column.embedding_column(from_to_year, 50),
    tf.feature_column.embedding_column(from_to_year_week, 50),
    tf.feature_column.embedding_column(weekday_year, 50),
    tf.feature_column.embedding_column(hour_weekday_year, 50),
    tf.feature_column.embedding_column(xy_pair, 50),

    # indicator columns
    tf.feature_column.indicator_column(loc_from),
    tf.feature_column.indicator_column(loc_to),
    tf.feature_column.indicator_column(hour),
    tf.feature_column.indicator_column(weekday),
    tf.feature_column.indicator_column(week),
    tf.feature_column.indicator_column(year),
    tf.feature_column.indicator_column(xy_0),
    tf.feature_column.indicator_column(xy_1),

    # numeric columns
    taxicab, euclidean, p_count
]

In [23]:
def train_input_fn(features, labels, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True,
        batch_size = batch_size)

def eval_input_fn(features, labels):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True)

def pred_input_fn(features):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        num_epochs = 1,
        shuffle = False)

In [24]:
model_dir = './DNN_Regressor_regions_coords'

file_writer = tf.summary.FileWriter(model_dir)

estimator = tf.estimator.DNNRegressor(
    model_dir = model_dir,
    feature_columns = deep_columns,
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001),
    hidden_units = [128,128,128,128], 
    batch_norm = True,
    dropout = 0.1,
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './DNN_Regressor_regions_coords', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001BE815DA9B0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn(X_train, y_train, batch_size=2500))
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn(X_eval, y_eval))

In [30]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_regions_coords\model.ckpt-16108
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 16108 into ./DNN_Regressor_regions_coords\model.ckpt.
INFO:tensorflow:loss = 30194.557, step = 16108
INFO:tensorflow:global_step/sec: 8.6369
INFO:tensorflow:loss = 25876.074, step = 16208 (11.580 sec)
INFO:tensorflow:global_step/sec: 9.61029
INFO:tensorflow:loss = 34284.56, step = 16308 (10.405 sec)
INFO:tensorflow:global_step/sec: 9.5788

INFO:tensorflow:global_step/sec: 9.50374
INFO:tensorflow:loss = 26213.277, step = 22508 (10.522 sec)
INFO:tensorflow:global_step/sec: 9.40063
INFO:tensorflow:loss = 21847.14, step = 22608 (10.638 sec)
INFO:tensorflow:global_step/sec: 9.6348
INFO:tensorflow:loss = 23740.188, step = 22708 (10.379 sec)
INFO:tensorflow:global_step/sec: 9.58202
INFO:tensorflow:loss = 25511.809, step = 22808 (10.436 sec)
INFO:tensorflow:global_step/sec: 9.65618
INFO:tensorflow:loss = 30054.266, step = 22908 (10.356 sec)
INFO:tensorflow:global_step/sec: 9.65198
INFO:tensorflow:loss = 30716.066, step = 23008 (10.361 sec)
INFO:tensorflow:global_step/sec: 9.60701
INFO:tensorflow:loss = 30860.877, step = 23108 (10.409 sec)
INFO:tensorflow:global_step/sec: 9.65542
INFO:tensorflow:loss = 31616.996, step = 23208 (10.368 sec)
INFO:tensorflow:global_step/sec: 9.62847
INFO:tensorflow:loss = 22831.246, step = 23308 (10.374 sec)
INFO:tensorflow:global_step/sec: 9.44096
INFO:tensorflow:loss = 24270.277, step = 23408 (10.5

INFO:tensorflow:loss = 28612.672, step = 29508 (10.321 sec)
INFO:tensorflow:global_step/sec: 9.64917
INFO:tensorflow:loss = 22015.496, step = 29608 (10.364 sec)
INFO:tensorflow:global_step/sec: 9.58202
INFO:tensorflow:loss = 26896.562, step = 29708 (10.436 sec)
INFO:tensorflow:global_step/sec: 9.65458
INFO:tensorflow:loss = 35963.867, step = 29808 (10.357 sec)
INFO:tensorflow:global_step/sec: 9.65766
INFO:tensorflow:loss = 39261.367, step = 29908 (10.355 sec)
INFO:tensorflow:global_step/sec: 9.68073
INFO:tensorflow:loss = 29563.07, step = 30008 (10.330 sec)
INFO:tensorflow:global_step/sec: 9.6653
INFO:tensorflow:loss = 17887.344, step = 30108 (10.346 sec)
INFO:tensorflow:global_step/sec: 9.66646
INFO:tensorflow:loss = 23346.57, step = 30208 (10.345 sec)
INFO:tensorflow:global_step/sec: 9.66118
INFO:tensorflow:loss = 29313.406, step = 30308 (10.363 sec)
INFO:tensorflow:global_step/sec: 9.64063
INFO:tensorflow:loss = 23002.932, step = 30408 (10.360 sec)
INFO:tensorflow:global_step/sec: 9

({'average_loss': 25.09404,
  'label/mean': 11.143269,
  'loss': 3212.037,
  'prediction/mean': 10.784786,
  'global_step': 32216},
 [])

In [27]:
infer = pd.read_csv('../input/test_processed.csv', index_col='key')

generator = estimator.predict(input_fn=pred_input_fn(infer))
predictions = [next(generator) for i in range(len(infer))]
values = [val['predictions'].tolist()[0] for val in predictions]

ids = infer.index
submission = pd.DataFrame()
submission['fare_amount'] = values
submission['key'] = ids
submission.set_index('key', inplace=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_regions_coords\model.ckpt-16108
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [28]:
submission.head()

,fare_amount
key,
2015-01-27 13:08:24.0000002,11.202897
2015-01-27 13:08:24.0000003,10.942987
2011-10-08 11:53:44.0000002,4.806397
2012-12-01 21:12:12.0000002,8.410332
2012-12-01 21:12:12.0000003,16.277964


In [29]:
submission.to_csv('../output/19.dnn_regressor_rmse_3.35.csv')